In [39]:
import torch
import pandas as pd

from transformers import RobertaTokenizerFast, RobertaForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

In [4]:
# Replace this location with where the repo is.
repository_location = "/Users/ameliachu/repos/nlu-reddit-toxicity-dataset"
interrater_data_folder = f"{repository_location}/data/interrater-reliability"

In [28]:
dataset = pd.read_csv(f"{interrater_data_folder}/interrater2-yj2369-20220410.csv")

In [14]:
dataset['inputs'] = dataset.apply(lambda x: f"{x['preceding_comment']} [SEP] {x['comment_for_evaluation']} [SEP] {x['following_comment']}", axis=1)

In [24]:
pretrained_model_name = "unitary/unbiased-toxic-roberta"

In [26]:
tokenizer = RobertaTokenizerFast.from_pretrained(pretrained_model_name, sep_token="[SEP]")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [32]:
dataset['encoded'] = dataset.apply(lambda row: tokenizer(row.inputs, padding='max_length'),
                                       axis=1)
dataset_encoded = dataset['encoded'].apply(pd.Series)

input_ids = torch.tensor(dataset_encoded["input_ids"].values.tolist())
attention_masks = torch.tensor(dataset_encoded["attention_mask"].values.tolist())
   


In [21]:
tokenizer(train_df['inputs'].iloc[0])

{'input_ids': [0, 627, 275, 86, 7, 907, 21, 23, 1273, 6, 5, 200, 275, 86, 7, 907, 16, 235, 122, 4, 50118, 1437, 50140, 8366, 42, 16, 10, 10297, 98, 5673, 99, 47, 32, 2882, 7, 2217, 50265, 42204, 5433, 12471, 37049, 24566, 347, 6934, 8827, 22062, 6178, 15004, 2562, 43499, 14079, 381, 42260, 38, 3326, 50265, 698, 330, 38, 1017, 28, 441, 7, 907, 10, 790, 8, 278, 62, 127, 3832, 4, 38, 1017, 33, 7, 173, 13, 277, 291, 50, 98, 107, 53, 38, 64, 697, 19, 14, 4, 50265], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [7]:
label_list = ['toxicity', 'severe_toxicity', 'identity_attack',
       'insult', 'profanity', 'threat']
# labels = dataset['label'].apply(lambda x: 1 if x else 0).tolist()

In [42]:
def compute_metrics(eval_pred):
    """Computes accuracy, f1, precision, and recall from a 
    transformers.trainer_utils.EvalPrediction object.
    """
    labels = eval_pred.label_ids
    preds = eval_pred.predictions.argmax(-1)

    ## TODO: Return a dictionary containing the accuracy, f1, precision, and recall scores.
    ## You may use sklearn's precision_recall_fscore_support and accuracy_score methods.
    accuracy = accuracy_score(labels, preds)
    precision, recall, f1_score, _ = precision_recall_fscore_support(labels, preds, beta=1)
    scores = {'accuracy': accuracy, 'f1': f1_score, 'precision': precision, 'recall': recall}
    return scores


def model_init():
    """Returns an initialized model for use in a Hugging Face Trainer."""
    # TODO: Return a pretrained RoBERTa model for sequence classification.
    # See https://huggingface.co/transformers/model_doc/roberta.html#robertaforsequenceclassification.
    model = RobertaForSequenceClassification.from_pretrained(pretrained_model_name)
    return model
training_args = TrainingArguments(**{'output_dir':'./model_outputs',
                'seed':42})
trainer_options = {
    "args": training_args,
    "model_init": model_init,
    "tokenizer": tokenizer,
    "compute_metrics": compute_metrics
}

trainer = Trainer(**trainer_options)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
loading configuration file https://huggingface.co/unitary/unbiased-toxic-roberta/resolve/main/config.json from cache at /Users/ameliachu/.cache/huggingface/transformers/ba294641f29d003c50875db9f8a1265f0bdf331683471246fc41395b6627ab00.667aa5ee4e5b6162fc4f5c749d4cafbde03dcd5ae1ef8ae9de8b80a66ea0113a
Model config RobertaConfig {
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "function_to_apply": "sigmoid",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "toxicity",
    "1": "severe_t

In [16]:
predictions = trainer.predict(tokenized_datasets["validation"])

"the best time to buy was at opening, the second best time to buy is right now.\n \n\n naturally this is a casino so bet what you are willing to lose[SEP]WORDS CANNOT DESCRIBE HOW SPECTACULARLY ERECT I AM[SEP]10k I'd be able to buy a house and set up my retirement. I'd have to work for another 20 or so years but I can live with that."

In [47]:
predictions = trainer.predict(dataset["encoded"])

***** Running Prediction *****
  Num examples = 6
  Batch size = 8


IndexError: index out of range in self

In [ ]:
    labels = eval_pred.label_ids
    preds = eval_pred.predictions.argmax(-1)

    ## TODO: Return a dictionary containing the accuracy, f1, precision, and recall scores.
    ## You may use sklearn's precision_recall_fscore_support and accuracy_score methods.
    accuracy = accuracy_score(labels, preds)
    precision, recall, f1_score, _ = precision_recall_fscore_support(labels, preds, beta=1)
    scores = {'accuracy': accuracy, 'f1': f1_score, 'precision': precision, 'recall': recall}
    return scores